# Model Comparisons Gemini / OpenAI

In [ ]:
!pip install trulens_eval==0.19.1 llama_index>=0.9.15post2 openai
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip install -q -U google-generativeai
!pip3 install gemini

### Step 1: Setup and Imports
First, ensure you have all the necessary libraries and API keys set up. You've already shown the setup for OpenAI and Hugging Face, so I'll focus on the setup for Google's Vertex AI Gemini.

In [ ]:
import os
import openai
from trulens_eval import Feedback, Tru, OpenAI, TruBasicApp, FeedbackMode
from trulens_eval.feedback.provider.hugs import (
    positive_sentiment,
    language_match,
    not_toxic,
    pii_detection
)
import requests  # For sending requests to Vertex AI


# Set up API keys
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path_to_your_google_credentials.json"

# Initialize OpenAI
openai.api_key = os.environ["OPENAI_API_KEY"]

# Initialize TruLens
tru = Tru()
tru.reset_database()


### Step 2: Define Functions for Models
Define functions for each model (GPT-3.5, GPT-4, and Gemini) that you want to compare.

In [ ]:
def gpt_model(prompt, model_name="gpt-3.5-turbo"):
    return openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a question and answer bot. Answer upbeat."},
            {"role": "user", "content": prompt}
        ]
    )["choices"][0]["message"]["content"]

def gemini_model(prompt):
    # Replace with the appropriate URL and project details for Vertex AI Gemini
    url = "https://us-central1-aiplatform.googleapis.com/v1/projects/YOUR_PROJECT_ID/locations/us-central1/publishers/google/models/gemini-pro:streamGenerateContent"
    headers = {
        "Authorization": "Bearer $(gcloud auth print-access-token)",
        "Content-Type": "application/json; charset=utf-8"
    }
    data = {
        "contents": {
            "role": "user",
            "parts": {"text": prompt}
        },
        # Add other configurations as needed
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()  # Parse and return the relevant part of the response


### Step 3: Initialize Feedback Functions
Set up the feedback functions as you have in your initial setup.

In [ ]:
f_language_match = Feedback(language_match, feedback_mode=FeedbackMode.DEFERRED).on_input_output()
f_positive_sentiment = Feedback(positive_sentiment, feedback_mode=FeedbackMode.DEFERRED).on_output()
f_not_toxic = Feedback(not_toxic, feedback_mode=FeedbackMode.DEFERRED).on_output()
f_pii_detection = Feedback(pii_detection, feedback_mode=FeedbackMode.DEFERRED).on_output()

### Step 4: Instrument the Callables for Logging with TruLens
Create TruBasicApp instances for each model.

In [ ]:
feedbacks=[
    f_language_match, f_positive_sentiment, f_not_toxic, f_pii_detection
]
gpt35_recorder = TruBasicApp(lambda p: gpt_model(p, "gpt-3.5-turbo"), app_id="gpt-3.5-turbo", feedbacks=feedbacks)
gpt4_recorder = TruBasicApp(lambda p: gpt_model(p, "gpt-4"), app_id="gpt-4", feedbacks=feedbacks)
gemini_recorder = TruBasicApp(gemini_model, app_id="gemini-pro", feedbacks=feedbacks)

### Step 5: Run Evaluations
Run the evaluations for each model using your prompts.

In [ ]:
prompts = [
    # Get Prompt List here evaluation/baselineprompts

with gpt35_recorder as recording:
    for prompt in prompts:
        gpt35_recorder.app(prompt)

with gpt4_recorder as recording:
    for prompt in prompts:
        gpt4_recorder.app(prompt)

with gemini_recorder as recording:
    for prompt in prompts:
        gemini_recorder.app(prompt)


### Step 5: Explore Results
Finally, use TruLens to explore the results of your evaluation.

In [ ]:
# Assuming you have TruLens setup
tru.run_dashboard()